In [ ]:
import pandas as pd
import numpy as np
import pickle as pkl

from sklearn import linear_model
from sklearn.metrics import mean_squared_error as mse

from bokeh.plotting import figure, show, output_notebook
from bokeh.charts import Scatter, show
output_notebook()

with open('dataframe.pkl', 'rb') as f:
    df = pkl.load(f)
    
models = ['regr', 'regr_ey', 'regr_log', 'nb1NN', 'nb2NN', 'nb3NN']

In [ ]:
def plot_coef(model, title):
# fun for plotting dataframe created by script fit_model.py
# model - name of column in df
    X = [x.intercept_ for x in df[model].values]
    Y= [y.coef_[0] for y in df[model].values]
    
    f = figure(title=title)
    f.scatter(x=X, y=Y, color='navy', size=6, alpha=.3)
    show(f)
    
def mse_count(model):
    X = [x*x for x in df[model+' d_A'].values]
    Y = [y*y for y in df[model+' d_B'].values]
    mse = 0
    for (x,y) in zip(X,Y):
        mse += np.sqrt(x*x + y*y)/2
    return mse/len(X)

In [ ]:
for  model in df.columns[:6]:
    plot_coef(model, 'intercept / coef '+model)

In [ ]:
for model in models:
    print(model, ' mse ', mse_count(model))